# Protein-ligand MD simulation

Note, because conda is essential for openmm, you need to set parameters, then `Run All` twice (condacolab does a kernel restart.)

The default simulation ([4O75+2RC](https://www.rcsb.org/3d-view/4O75/1),
plus the LEU92HIS mutation)
takes around 25m on a T4 GPU.

colab by [@btnaughton](https://twitter.com/btnaughton)

In [ ]:
#@title MD_protein_ligand PDB input

pdb_id = '4O75' #@param {type:"string"}
ligand_id = '2RC' #@param {type:"string"}
#@markdown Since the ligand may be present on multiple chains, we must specify
ligand_chain = "A" #@param {type:"string"}
#@markdown Simulate solvent. Much much slower!
use_solvent = False #@param {type:"boolean"}
#@markdown Download a tar file containing all results?
download_results = True #@param {type:"boolean"}
#@markdown If minimize_only is True, then num_steps does nothing
minimize_only = True #@param {type:"boolean"}
num_steps = 0 #@param {type:"number"}

#@markdown ## mutation
#@markdown Optionally, mutate a position
#@markdown (e.g., mutate "LEU" at residue 92 to "HIS" on chains "A" or "AB")
mutate_from = "LEU" #@param {type:"string"}
mutate_resn = 92 #@param {type:"number"}
mutate_to = "HIS" #@param {type:"string"}
mutate_chains = "A" #@param {type:"string"}

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
!mamba install -q openmm=8.0.0 openmmforcefields=0.11.2 pdbfixer=1.9 rdkit=2023.03.1 mdtraj=1.9.9 plotly=4.9.0 openff-toolkit=0.14.3 python-kaleido=0.2.1 mdanalysis=2.5.0 prody=2.4.0

In [ ]:
!pip install git+https://github.com/hgbrian/MD_protein_ligand --quiet
!pip install py3dmol==2.0.3 --quiet
!pip install ipython-autotime --quiet
%load_ext autotime

## Example 1
We have a PDB file containing a ligand, minimize it.

smina is faster than gnina, but use gnina for more serious cases

In [ ]:
from pathlib import Path
from shutil import copy
from MD_protein_ligand.simulate import get_pdb_and_extract_ligand, simulate

# defaults / less common options
use_pdb_redo = True

SIM_PARAMS = dict(num_steps=num_steps,
                  use_solvent=use_solvent, decoy_smiles=None, minimize_only=minimize_only,
                  temperature=310, equilibration_steps=200,
                  scoring_tool='smina')

out_dir = f"MD_protein_ligand/{pdb_id}_{ligand_id}"

prepared_files = get_pdb_and_extract_ligand(pdb_id, ligand_id,
                                            out_dir=out_dir, use_pdb_redo=use_pdb_redo, ligand_chain=ligand_chain)

sim_files = simulate(prepared_files["pdb"], prepared_files["sdf"], f"{out_dir}/{pdb_id}_{ligand_id}",
                     **SIM_PARAMS)
print(f"{sim_files=}")
print("-"*100)

# mutate a residue and relax again!
if all(param for param in (mutate_from, mutate_resn, mutate_to, mutate_chains)):
    mutated_file = f"{out_dir}/{Path(prepared_files['pdb']).stem}_{mutate_chains}_{mutate_from}{mutate_resn}{mutate_to}"
    copy(prepared_files["pdb"], f"{mutated_file}.pdb")

    mutated_prepared_files = get_pdb_and_extract_ligand(f"{mutated_file}.pdb", out_dir=out_dir,
                                                        use_pdb_redo=use_pdb_redo, mutation=(mutate_from, mutate_resn, mutate_to, mutate_chains))
    print(f"{mutated_prepared_files=}")

    mutated_sim_files = simulate(mutated_prepared_files["pdb"], prepared_files["sdf"], mutated_file,
                                 **SIM_PARAMS)

In [ ]:
from google.colab import files
from datetime import datetime
import locale
locale.getpreferredencoding = lambda: "UTF-8" # shrug

if download_results:
    tarname = f"MD_protein_ligand_{datetime.now().isoformat()[2:10].replace('-','')}"
    _ = !tar cvf {tarname}.tar out

    files.download(f"{tarname}.tar")

Examine the difference in affinity between the complex (original PDB file) and the new minimized version.

Optionally, compare this to mutated residues too.

In [ ]:
import pandas as pd

df_aff = pd.read_csv(sim_files["smina_affinity_tsv"], sep='\t')
if all(param for param in (mutate_from, mutate_resn, mutate_to, mutate_chains)):
    df_aff = pd.concat([df_aff.assign(mutation=''),
                        pd.read_csv(mutated_sim_files["smina_affinity_tsv"], sep='\t').assign(mutation=f"{mutate_from}_{mutate_resn}_{mutate_to}")])

df_aff

In [ ]:
import py3Dmol

resid_hover = """
function(atom,viewer) {
    if(!atom.label) {
        atom.label = viewer.addLabel(atom.chain+" "+atom.resn+" "+atom.resi,
            {position: atom, backgroundColor: 'mintcream', fontColor:'black', fontSize:12});
    }
}"""
unhover_func = """
function(atom,viewer) {
    if(atom.label) {
        viewer.removeLabel(atom.label);
        delete atom.label;
    }
}"""

view = py3Dmol.view(width=800, height=800)
view.setCameraParameters({'fov': 35, 'z': 100});

# add sdf
view.addModel(open(sim_files['complex_pdb']).read(), "pdb")
view.setStyle({"model": 0}, {"cartoon":{"color":"#666666"}})
view.setStyle({"model": 0, "hetflag":True}, {'stick':{"color":"#666666"}})

# add pdb
view.addModel(open(sim_files['minimized_pdb']).read(), "pdb");
view.setStyle({"model": 1}, {"cartoon":{"color":"#df3964"}})
view.setStyle({"model": 1, "hetflag":True}, {'stick':{"color":"#df4656"}})
view.zoomTo();

model = view.getModel()
model.setHoverable({}, True, resid_hover, unhover_func)

view